<a href="https://colab.research.google.com/github/sinhvienfpt/UsedCarPricePrediction/blob/develop/OldCarPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
url = "https://raw.githubusercontent.com/sinhvienfpt/UsedCarPricePrediction/main/assets/data/train-data.csv"

In [159]:
import pandas as pd
pd.options.mode.chained_assignment = None #Ignore pandas warning

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

## Data exploring

In [160]:
df = pd.read_csv(url,index_col=0)
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [161]:
print(df.shape)

(6019, 13)


### Data Exploration

### `Year`
Add an column `age` calculated by subtracting this year (2024) from the `year`

In [162]:
df["age"] = 2024 - df["Year"]
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,age
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75,14
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,9
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,13
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,12
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,11


#### `Mileage`
The `Mileage` column in your data uses two different units of measurement: 'km/kg' and 'kmpl'.

In [163]:
kmkg = 0
kmpl = 0
for i in df.Mileage:
    if str(i).endswith("km/kg"):
        kmkg+=1
    elif str(i).endswith("kmpl"):
        kmpl+=1
print('The number of rows with Km/Kg : {} '.format(kmkg))
print('The number of rows with Kmpl : {} '.format(kmpl))

The number of rows with Km/Kg : 66 
The number of rows with Kmpl : 5951 


So we can see the amount of rows with Kmpl is greater than Km/Kg (nearly 90 times). So I decided to convert from Km/Kg into Kmpl. Let take a view at the fuel_type which use km/kg for Mileage.

In [164]:
km_kg_fuel_type = set()
for i in range(len(df)):
    if str(df.Mileage[i]).endswith("km/kg"):
        km_kg_fuel_type.add(df.Fuel_Type[i])
print(km_kg_fuel_type)

{'LPG', 'CNG'}


`CNG` (Compressed Natural Gas) and `LPG` (Liquefied Petroleum Gas) are two types of gas fuels that use km/kg units in the data.

| Characteristic | CNG | LPG |
|----------------|-----|-----|
| Density | 0.75 - 0.83 Kg/Sm³ | 0.51 - 0.56 kg/L |
|Average Desity| 0.79 Kg/Sm³| 0.535 kg/L|

In [165]:
# Now convert
for i in range(len(df)):
    if str(df.Mileage[i]).endswith("km/kg"):
        if df.Fuel_Type[i] == "CNG":
            df.Mileage[i] = float(df.Mileage[i].split()[0])*0.79
        elif df.Fuel_Type[i] == "LPG":
            df.Mileage[i] = float(df.Mileage[i].split()[0])*0.535
    elif str(df.Mileage[i]).endswith("kmpl"):
        df.Mileage[i] = float(df.Mileage[i].split()[0])
    else:
        df.Mileage[i] = np.nan
df.head()


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,age
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,21.014,998 CC,58.16 bhp,5.0,NaN,1.75,14
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582 CC,126.2 bhp,5.0,NaN,12.50,9
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,13
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248 CC,88.76 bhp,7.0,NaN,6.00,12
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968 CC,140.8 bhp,5.0,NaN,17.74,11


#### `Engine` and `Power`

In [166]:
# Find the units of columns Power
tmp = df["Power"].str.split(" ").str[1].unique()
print(tmp)

# Find the units of columns Engine
tmp = df["Engine"].str.split(" ").str[1].unique()
print(tmp)

['bhp' nan]
['CC' nan]


In [167]:
# Convert them  to numeric
def safe_float(x):
    try:
        return float(x.split()[0])
    except:
        return np.nan

df["Power"] = df["Power"].apply(safe_float)
df["Engine"] = df["Engine"].apply(safe_float)

df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,age
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,21.014,998.0,58.16,5.0,NaN,1.75,14
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,12.50,9
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199.0,88.70,5.0,8.61 Lakh,4.50,13
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,6.00,12
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968.0,140.80,5.0,NaN,17.74,11


#### Category variables

In [168]:
df["Location"].unique()

array(['Mumbai', 'Pune', 'Chennai', 'Coimbatore', 'Hyderabad', 'Jaipur',
       'Kochi', 'Kolkata', 'Delhi', 'Bangalore', 'Ahmedabad'],
      dtype=object)

The column `Location` has 10 unique values, that's too much for One Hot encoding

In [169]:
df["Fuel_Type"].unique()

array(['CNG', 'Diesel', 'Petrol', 'LPG', 'Electric'], dtype=object)

In [170]:
# One hot
ohe = OneHotEncoder(sparse=False)
ohe.fit(df[["Fuel_Type"]])
ohe.categories_

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[array(['CNG', 'Diesel', 'Electric', 'LPG', 'Petrol'], dtype=object)]

In [171]:
ohe.transform(df[["Fuel_Type"]])
df[ohe.categories_[0]] = ohe.transform(df[["Fuel_Type"]])
df.head()


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,age,CNG,Diesel,Electric,LPG,Petrol
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,21.014,998.0,58.16,5.0,NaN,1.75,14,1.0,0.0,0.0,0.0,0.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,12.50,9,0.0,1.0,0.0,0.0,0.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199.0,88.70,5.0,8.61 Lakh,4.50,13,0.0,0.0,0.0,0.0,1.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,6.00,12,0.0,1.0,0.0,0.0,0.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968.0,140.80,5.0,NaN,17.74,11,0.0,1.0,0.0,0.0,0.0


In [172]:
df["Transmission"].unique()

array(['Manual', 'Automatic'], dtype=object)

In [173]:
# One hot
ohe = OneHotEncoder(sparse=False)
ohe.fit(df[["Transmission"]])
ohe.categories_

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[array(['Automatic', 'Manual'], dtype=object)]

In [174]:
ohe.transform(df[["Transmission"]])
df[ohe.categories_[0]] = ohe.transform(df[["Transmission"]])
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,...,New_Price,Price,age,CNG,Diesel,Electric,LPG,Petrol,Automatic,Manual
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,21.014,998.0,58.16,...,NaN,1.75,14,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,...,NaN,12.50,9,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199.0,88.70,...,8.61 Lakh,4.50,13,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,...,NaN,6.00,12,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968.0,140.80,...,NaN,17.74,11,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [175]:
df["Owner_Type"].unique()

array(['First', 'Second', 'Fourth & Above', 'Third'], dtype=object)

In [176]:
# Ondinary decoding
df["Owner_Type"] = df["Owner_Type"].map({"First":1,"Second":2,"Third":3,"Fourth & Above":4})
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,...,New_Price,Price,age,CNG,Diesel,Electric,LPG,Petrol,Automatic,Manual
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,1,21.014,998.0,58.16,...,NaN,1.75,14,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,1,19.67,1582.0,126.20,...,NaN,12.50,9,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,1,18.2,1199.0,88.70,...,8.61 Lakh,4.50,13,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,1,20.77,1248.0,88.76,...,NaN,6.00,12,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,2,15.2,1968.0,140.80,...,NaN,17.74,11,0.0,1.0,0.0,0.0,0.0,1.0,0.0


**Drop no-need columns**

In [177]:
cols_to_drop = ["Year","Fuel_Type","Transmission","Owner_Type"]
df = df.drop(cols_to_drop,axis=1)
df.head()

,Name,Location,Kilometers_Driven,Mileage,Engine,Power,Seats,New_Price,Price,age,CNG,Diesel,Electric,LPG,Petrol,Automatic,Manual
0,Maruti Wagon R LXI CNG,Mumbai,72000,21.014,998.0,58.16,5.0,NaN,1.75,14,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,41000,19.67,1582.0,126.20,5.0,NaN,12.50,9,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Honda Jazz V,Chennai,46000,18.2,1199.0,88.70,5.0,8.61 Lakh,4.50,13,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,Maruti Ertiga VDI,Chennai,87000,20.77,1248.0,88.76,7.0,NaN,6.00,12,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,40670,15.2,1968.0,140.80,5.0,NaN,17.74,11,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Save it to new csv file

In [179]:
df.to_csv("cleaned_data.csv")